# Local Homology

In this tutorial we use tools from Topological Data Analysis (TDA) to study the local geometry of point clouds.

If you are looking at a static version of this notebook and would like to run its contents, head over to [GitHub](https://github.com/giotto-ai/giotto-tda/blob/master/examples/local_homology.ipynb) and download the source.

## See also

- [Topological feature extraction using VietorisRipsPersistence and PersistenceEntropy](https://giotto-ai.github.io/gtda-docs/latest/notebooks/vietoris_rips_quickstart.html) for a quick introduction to general topological feature extraction in ``giotto-tda``.
- [Local Homology NLP Use Cases: Unsupervised text disambiguation](https://giotto-ai.github.io/gtda-docs/latest/notebooks/local_hom_NLP_disambiguation.html), in which the ideas presented here are applied to natural language data!

**License: AGPLv3**

In [ ]:
import numpy as np

from gtda.plotting import plot_point_cloud
from gtda.local_homology import KNeighborsLocalVietorisRips, RadiusLocalVietorisRips

# Dimensionality

## What is "dimensionality", and how can we formalize it? 

In this tutorial we will learn how to algorithmically distinguish between possible *local geometries* around data points. Consider, for instance, the local geometry around the red dot in the three shapes below:

![locally_different](images/locally_different.png)

## How?

We propose an answer to this question based on the notion of **local homology**. We estimate the "local dimension" around each point by studying the topological properties of the data points' neighbourhoods. We start by studying three shapes which differ by their dimensions, namely the line, the plane, and the cube.

In [ ]:
n = 10
line_1d = np.linspace(0, 1, num=n, endpoint=False)

line = np.c_[np.zeros((n, 2)), line_1d]
plane = np.array([[0, y, z] for y in line_1d for z in line_1d])
cube = np.array([[x, y, z] for x in line_1d for y in line_1d for z in line_1d])

line_plane_cube = np.vstack((line, plane + [0.5, -0.5, 0], cube + [1, -0.5, 0]))
plot_point_cloud(line_plane_cube)

As explained in [Topological feature extraction using VietorisRipsPersistence and PersistenceEntropy](https://giotto-ai.github.io/gtda-docs/latest/notebooks/vietoris_rips_quickstart.html), "persistence diagrams" are a common and useful way to store information about the topology and geometry of data. Their content can be summarized and made interpretable even further by means of a variety of featurization methods. In this notebook, we use a modified version of the "persistent entropy" computed by the ``PersistenceEntropy`` transformer.

In [ ]:
from gtda.diagrams import PersistenceEntropy
from sklearn.preprocessing import FunctionTransformer
from gtda.pipeline import make_pipeline

modified_persistence_entropy = make_pipeline(PersistenceEntropy(),
                                             FunctionTransformer(func=lambda X: 2 ** X))

def plot_coloured_cloud(local_hom, diags, point_cloud, dimension):
    """This is a custom plotting function, that plots the point cloud and colours 
     the points according to their persistence diagrams in the given dimension."""
    dim_index = np.argwhere(np.array(local_hom.homology_dimensions) == dimension)[0][0]
    dim_vects = modified_persistence_entropy.fit_transform(diags)
    return plot_point_cloud(point_cloud,
                            plotly_params={"trace" : {"marker": {"color": dim_vects[:, dim_index],
                                                                 "size": 6,
                                                                 "showscale": True},
                                                      "text": [f"color: {i}" for i in dim_vects[:, dim_index]]}})

## The method

To compute and represent the dimensionality at each point we do the following:
- For each point in the data set we find a vector $(d_1, \dots, d_i)$ where $d_k$ correponds to $k$-dimensional features. The $i$-th dimensional feature correspond to the $i$-th Betti number of the "coned-off" local neighborhood around that point. More on this in "How does it work" below!
- We then plot the point cloud by colouring the points with one of the computed features.


In [ ]:
# First initialize a local homology transformer. 
# The notion of neighborhood here is determined by the number of closest neighbors to consider
lh_line = KNeighborsLocalVietorisRips(n_neighbors=(1, 7))

# fit_tranform the transformer to get the persistence diagrams
diags = lh_line.fit_transform(line)

# plot the results of local homology dimension 1:
plot_coloured_cloud(lh_line, diags, line, 1)

In the example above, for each non-boundary point, the algorithm only detects one local feature, in dimension 1!

In [ ]:
# plane coloured with two dimensional features
# the notion of neighborhood here is determined by radii.
lh_plane = RadiusLocalVietorisRips(radii=(0.001, 0.3),
                                   homology_dimensions=(1, 2, 3))

# Fit-transform
diags = lh_plane.fit_transform(plane)

# Plot the results of local homology dimension 2:
plot_coloured_cloud(lh_plane, diags, plane, 2)

In the two examples above we plotted the one dimensional features of the line, and the two dimensional features of the plane. Both are equal to 1, except at the borders/boundaries. 

## Non homogeneous example

In the following example, we  consider a point cloud with varying dimensionality. We plot the three shapes next to each other. We encourage you to play around with the dimension parameter in the plotting function below, to compare the local structure of points in the line, plane and cube! One sees that one dimensional features are observed in all three shapes, but two dimensional features only exist in the plane and three dimensional features only exist in the cube.

In [ ]:
# Here that we set the `collapse_edges` parameter to True,
# which allows the algorithm to be paralellized and run faster!

lh_line_plane_cube = RadiusLocalVietorisRips(radii=(0.01, 0.25),
                                             homology_dimensions=(1, 2, 3),
                                             collapse_edges=True)



# Fit-transform
diags = lh_line_plane_cube.fit_transform(line_plane_cube)

In [ ]:
# Plot the results of local homology dimension 1. We encourage you to change the dimension parameter.
plot_coloured_cloud(lh_line_plane_cube, diags, line_plane_cube, dimension=1)

In [ ]:
# First generate the point cloud
line1=[[x, 0, 0] for x in np.arange(-1,1, 1/10)]
line2=[[0, y, 0] for y in np.arange(-1,1, 1/10)]
line3=[[x, 1, 0] for x in np.arange(-1,1, 1/10)]
lines=np.array(line1+line2+line3)

# Plot the point cloud
plot_point_cloud(lines)

Our last example is inherently one dimensional, but the point cloud admits some interesting local structures, there are 4 different types of points:
- The 'edge points', with no local structure
- The 'line points' which locally look like one dimensinoal lines.
- The 'intersection points', which locally looks like an X.
- The 'intersection points', which locally looks like a T.

The next block looks at the local homology features!

In [ ]:
# Initialize the radius local homology transformer
# Here we set the `n_jobs` parameter to -1, to allow the algorithm to distribute and run faster!
lh_lines = RadiusLocalVietorisRips(radii=(0.21, 0.31),
                                   homology_dimensions=(0, 1),
                                   n_jobs=-1)

# Fit_transform
diags = lh_lines.fit_transform(lines)

# Plot the results of local homology dimension 1:
plot_coloured_cloud(lh_lines, diags, np.array(lines), dimension=1)

Notice that the one-dimensional feature counts the branches at a given point!

We encourage you to change the values of the radii parameter. If you are curious of what these parameters are, please scroll down to the 'How idoes it work' section.

# How does it work?

As the mathematics behind this algorithm is quite involved, we give an explanation for one-dimensional examples. We also give a picture for the process for a 2 dimensional example.

A central theme in Topology is the study of properties that are invariant under deformation. A popular one-dimensional invariant is the number of 'loops' in the space. For a graph, the number of 'loops' is the number of cycles of that graph. In the case of connected graphs this is equal to $1-\chi(G)$, where $\chi(G)=|V|-|E|$ is the Euler characteristic of the graph, and $|V|$ and $|E|$ are the number of vertices and edges respectively. 

However, although all trees have Euler characteristic 1, they can admit very different local behaviour. The Euler characteristic alone will not be enough, we therefore have to do a trick:

The first step is to discard points that are far away, since they do not affect the local geometry around a point. We thus keep points that are closer than the second radius (either a distance from the point for the RadiusLocalVietrosiRips transformer, or the closest neighbors for the KNeighborsVietorisRips transformer).

The second step is the one making loops reflect dimensionality. From a line, how can we obtain a loop? We stick the two ends of the line together! This trick can be thought of as 'coning' off the endpoints: adding an extra point and linking some points (the furthest points) to this extra point. Our algorithm cones off the points that lie in between the first and second radii, as depicted below.

So what are the Euler characteristic of the resulting graph below?
It one has two vertices (at the intersection points) and 4 edges, so Euler characteristic -2! This relects the complexity at the point that we are considering! (Notice also that a loop has Euler characteristic 0, and a triple intersection will give Euler characteristic -1)

We also added a 2-dimensional example: the disk. By coning off the furthest point, we end up with a sphere! Which only has nontrivial 2 dimensional features.

![coning_off](images/coning_off.png)
<div style="text-align: left">
   <p style="text-align: center;">  Coning off a cross and a disk.</p>
</div>